In [ ]:
"""
plot the different instruments, (linestyles)
plot the different energy bands, (colors)
plot the 

currently not excluding the extension sources

# load the surface brightness of different parameters
# temperarily take 5.0-12.5 arcmin

## make_radialprofiles_rate.py

"""

In [ ]:
## onserver: plot_ctr-vs-pssize.py

import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import os
import seaborn as sns
sns.set_palette("colorblind")



def radial_prof(rad_bins, center_file, id, DATFILE, EXPFILE, NXBFILE, CHEESEFILE):
    """
    This function plot the radial profile from xmm individual instrument data

    """

    import numpy as np
    from astropy.io import fits
    import pandas as pd
    import os
    from astropy.wcs import WCS
    from astropy.coordinates import SkyCoord

    regf = pd.read_csv(center_file)


    ra = regf[regf['ID']==id]['RA']
    dec = regf[regf['ID']==id]['DEC']

    # convert the fk5 coords to pixel coordinates
    dat_widths = fits.getdata(DATFILE).shape
    x,y = np.meshgrid(range(0, dat_widths[0]), range(0, dat_widths[1]))
    wcs = WCS(fits.getheader(DATFILE))
    RA,DEC = wcs.pixel_to_world_values(x,y)
    distance = SkyCoord(ra=ra,dec=dec,unit='deg').separation(SkyCoord(ra=RA,dec=DEC,unit='deg')).arcmin
    # ax.imshow(distance)

    cts_map = fits.getdata(DATFILE)*fits.getdata(CHEESEFILE)
    bkg_map = fits.getdata(NXBFILE)*fits.getdata(CHEESEFILE)
    exp_map = fits.getdata(EXPFILE)*fits.getdata(CHEESEFILE)
    msk_map = fits.getdata(CHEESEFILE)

    CTS, r_bins = np.histogram(distance.flatten(),weights=cts_map.flatten(),bins=rad_bins)

    BKG, r_bins = np.histogram(distance.flatten(),weights=bkg_map.flatten(),bins=rad_bins)

    EXP, r_bins = np.histogram(distance.flatten(),weights=exp_map.flatten(),bins=rad_bins)

    Npixel, r_bins = np.histogram(distance.flatten(),weights=msk_map.flatten(),bins=rad_bins)
    

    return CTS, BKG, EXP, Npixel


ELOs = [500, 2000, 500]
EHIs = [2000, 8000, 8000]

linestyles = ['solid', 'dashed', 'dotted']
linewidths = [2,2,2]
alphas = [0.8, 0.8, 0.8]
markers = ['o', '>', 'x']
center_file = f'/data/yanling/XGAP-ABUN/data/alldata/XGAP/dats/all_centers.csv'
rootpath = f'/data/yanling/XGAP-ABUN/data/alldata/XGAP'

rbins = np.linspace(0,30,100)
contours = np.arange(0.05, 1.0, 0.05)


# for dir in glob(f'{rootpath}/SDSSTG11844'): 
    # id = dir.split('/')[-1]
for id in ['SDSSTG1162', 'SDSSTG12349', 'SDSSTG828']:
    dir = f'{rootpath}/{id}'
    print(id)
    datapath = f'{dir}/{id}_allfov/final_img'
    print(datapath)
    savepath = f'{rootpath}/figs/radial_profiles/{id}'
    os.makedirs(savepath, exist_ok=True)
    fig, axs = plt.subplots()
    for e_idx in range(len(ELOs)):

        for i, inst in enumerate(['mos1*', 'mos2*', 'pn*']):
            ctr = np.zeros(len(contours))
            ctr_err = np.zeros(len(contours))
            for j, contour in enumerate(contours):
                DATFILE = glob(f'{datapath}/{inst}-fovimsky-{ELOs[e_idx]}-{EHIs[e_idx]}.fits')[0]
                EXPFILE = glob(f'{datapath}/{inst}-expimsky-{ELOs[e_idx]}-{EHIs[e_idx]}.fits')[0]
                print(f'{datapath}/{inst}-bkgimsky-{ELOs[e_idx]}-{EHIs[e_idx]}.fits')
                NXBFILE = glob(f'{datapath}/{inst}-bkgimsky-{ELOs[e_idx]}-{EHIs[e_idx]}.fits')[0]
                CHEESEFILE = glob(f'{dir}/{inst}-cheeset_contour_{contour:.2f}_addext.fits')[0]

                CTS, BKG, EXP, N_PIX = radial_prof(rbins, center_file, id, DATFILE, EXPFILE, NXBFILE, CHEESEFILE)
                
                net = (CTS - BKG) / EXP
                neterr = np.sqrt(CTS - BKG) / EXP

                filt = (rbins[:-1] <= 12.5) & (rbins[:-1] >= 5)
                ctr[j] = np.nansum(net[filt])
                ctr_err[j] = np.nansum(neterr[filt])

            # norm the ctr
            ctr = ctr/np.nanmedian(ctr)
            # ctr_err = ctr_err / np.nanmedian(ctr_err)          
            # axs.errorbar(contour, ctr, yerr = ctr_err, color=sns.color_palette()[e_idx], linestyle =linestyles[i], linewidth = linewidths[i], alpha = alphas[i],  label = f'{inst[:-1]}_{ELOs[e_idx]}-{EHIs[e_idx]}')
            axs.plot(contours, ctr, marker = markers[i], color=sns.color_palette()[e_idx], linestyle =linestyles[i], linewidth = linewidths[i], alpha = alphas[i],  label = f'{inst[:-1]}_{ELOs[e_idx]}-{EHIs[e_idx]}')
            

    axs.set_title(f'{id}')
    # axs.set_xlabel(f'point src size [R(90% PSF)]')
    axs.set_xlabel(f'bkgfrac of contour command')
    axs.set_ylabel(f'norm count rate in annuli 5-12.5 arcmin [cts/s/pixel]')
    # axs.set_yscale('log')
    plt.legend()
    plt.tight_layout()
    plt.savefig(f'{savepath}/ctr_vs_ps-size_{id}_norm_linear_contour_vary_addext.png')
                
